In [1]:
import multiprocessing
multiprocessing.set_start_method('fork')


In [2]:
import warnings
warnings.filterwarnings('ignore', category=RuntimeWarning)

In [3]:
import collections
import random
from random import uniform as ru
from math import *
import scipy.optimize as opt

def H(c):
    """
    Entropy function
    """
    if c == 0. or c == 1.:
        return 0.
    
    if c < 0. or c > 1.:
        return -1000
    
    return -(c * log2(c) + (1 - c) * log2(1 - c))

def binomH(n,k):
    """
    binomial coefficient
    """
    # if k/n not in ZZ:
    #     return -100
    if n <=0:
        return 0
    return n * H(k/n)

def multiH(n,c):
    """
    multinomial coefficient
    """
    if sum(c)>n:
        if sum(c)-n>-10**(-8):
            c[-1]=n-sum(c[:-1])
        else:
            return 100
    tot=0
    val=n
    for i in c:
        if i<0 and i>(-10**(-9)):
            pass
        else:
            tot+=binomH(n,i)
            n-=i
    return tot

def reps(p, m, d, l): 
    """
    representations of length-l vector with p ones and m minus ones
    two length-l vectors with p/2+d ones and d minus ones each.
    """
    if p <= 0.000001 or l == 0.:
        return 0
    if l < p or l - p < 2*d:
        return 0.
    
    return l*p + l*m + multiH(l-p-m, [d,d]) 

def wrap(f,g) :
    def inner(x):
        return f(g(*x))
    return inner

In [4]:
def compute_list(m):
    L=[]
    i=0
    while i <50:
        j=0.01*i
        global M
        M= j*log2(2*m+1)

        res=do_parallel()
        if i!=0 and (res>L[-1][1]+0.0001 or res<L[-1][1]-0.05):
            continue
        L.append([j,res])
        i+=1
        print(j,res)
    return L

In [5]:
def r(x,y,z):
    return [(ru(x,y)) for i in range(z)]

 

In [6]:
def do_parallel():
    mini=1000
    candidates=[]
    with multiprocessing.Pool(PROCESSES) as pool:
        params = range(PROCESSES)
        results = [pool.apply_async(opti) for p in range(PROCESSES)]
        for i in results:

            candidates.append(i.get())
    mini=min(mini,min(candidates))
    # for i in candidates:
    #     if mini==1000 or i.fun<mini.fun:
    #         mini=i
    return mini

def time(x):
    x=set_vars(*x)
    return (D(x)/2+max(0,D(x)-R(x)-min(M/2,(D(x)-R(x))/2)))/log2(2*m+1)

def opti():
    mini=[1000]
    for i in range(ITERATIONS_PER_PROCESS):
        start= r(0,0.2,number_opt_params-1)+r(0,0.999,1)
        bounds=[(0.0,1/2-w([]))]*(number_opt_params-1)+[(0,1)]
        result = opt.minimize(time, start, 
                bounds= bounds, tol=1e-10, 
                constraints=constraints_csidh, options={'maxiter':10000})
        if result.fun<mini[0] and result.success and result.fun>0:
            mini=[result.fun,result]
        
    return mini[0]

### Standard Rep-1 VoW with / without partial representations

In [7]:
PROCESSES = 4
ITERATIONS_PER_PROCESS=100


# x.e corresponds to epsilon
# x.d corresponds to delta
set_vars = collections.namedtuple('CSIDH', 'e d')
def csidh1(f) : return wrap(f, set_vars)

number_opt_params=2
m=1

# w denotes the proportion of +1 and -1 entries each, i.e., the proportion of zeros is 1-2w
# D is always the function domain size and R the amount of representations
w= lambda x: 1/3
D= lambda x: multiH(x.d,[x.d*(w(x)/2+x.e),x.d*(w(x)/2+x.e)])+multiH((1-x.d)/2,[(1-x.d)*w(x)/2,(1-x.d)*w(x)/2])
R = lambda x: reps(w(x),w(x),x.e,x.d)



In [8]:
constraints_csidh = [
  { 'type' : 'ineq',   'fun' : csidh1(lambda x : 1-(w(x)+2*x.e))},
  # uncomment for using no partial representations
  #{ 'type' : 'eq'  ,   'fun' : csidh(lambda x :1-x.d)}  
]

In [9]:
L_partial_reps=compute_list(m)

0.0 0.6747103788458106
0.01 0.6697103788458107
0.02 0.6647103788458106
0.03 0.6597103788458106
0.04 0.6547103788458107
0.05 0.6497103788458107
0.06 0.644710378845811
0.07 0.6397103788458107
0.08 0.6347103788458108
0.09 0.6297103788458108
0.1 0.6247103788458107
0.11 0.6197103788458107
0.12 0.6147103788458106
0.13 0.6097103788458107
0.14 0.6047103788458109
0.15 0.5997103788458106
0.16 0.5947103788458106
0.17 0.5897103788458107
0.18 0.5847103788458106
0.19 0.5797103788458106
0.2 0.5747103788458107
0.21 0.5697103788458107
0.22 0.5647103788458108
0.23 0.5608305680708254
0.24 0.5587726577395876
0.25 0.5573564741023243
0.26 0.5560625293447461
0.27 0.5548960385576323
0.28 0.5538609119183004
0.29 0.5529599482932315
0.3 0.5521950052689331
0.31 0.5515671491244843
0.32 0.5510767826290451
0.33 0.5507237584375273
0.34 0.5505074753714239
0.35000000000000003 0.5504269565185058
0.36 0.5504263240372096
0.37 0.5504263240372104
0.38 0.5504263240372136
0.39 0.5504263240372133
0.4 0.55
0.41000000000000003 0

### Increased Rep

In [10]:
PROCESSES = 4
ITERATIONS_PER_PROCESS=100

number_opt_params=4
m=1
set_vars = collections.namedtuple('CSIDH2', 'z1 z2 o d')
def csidh2(f) : return wrap(f, set_vars) 

def reps2(x):
    x=set_vars(*x)
    t=x.d*w(x)
    return multiH(x.d*(1-2*w(x)),[x.z2,x.z1,x.z1,x.z2])+2*multiH(t,[t/2-x.o,t/2-x.o,x.o])

In [11]:
w= lambda x: 1/3
a = lambda x: w(x)/2 +x.z1/x.d
b = lambda x: (x.z2+x.o)/x.d
z0 = lambda x:x.d*(1-2*w(x))-2*x.z1-2*x.z2

D= lambda x: multiH(x.d,[x.d*a(x),x.d*a(x),x.d*b(x),x.d*b(x)])+multiH((1-x.d)/2,[(1-x.d)*w(x)/2,(1-x.d)*w(x)/2])
R= lambda x: reps2(x)

In [12]:
constraints_csidh = [
  { 'type' : 'ineq',   'fun' : csidh2(lambda x : x.d*w(x)/2-x.o)},
  { 'type' : 'ineq',   'fun' : csidh2(lambda x : z0(x))},
  { 'type' : 'ineq',   'fun' : csidh2(lambda x : 1-(2*a(x)+2*b(x)))}
]


In [13]:
L_increased_rep=compute_list(m)

0.0 0.6703408387093247
0.01 0.6653408387093472
0.02 0.6603408387093104
0.03 0.6553408387093392
0.04 0.6503408387092958
0.05 0.6453408387092655
0.06 0.6403408387092996
0.07 0.6353408387093056
0.08 0.6303408387093395
0.09 0.625340838709278
0.1 0.6203408387092625
0.11 0.615340838709265
0.12 0.6103408387092955
0.13 0.605340838709283
0.14 0.6003408387092769
0.15 0.5953408387092638
0.16 0.5903408387093206
0.17 0.5853408387093079
0.18 0.5803408387093084
0.19 0.5753439359577663
0.2 0.5707257423231611
0.21 0.5667323243175726
0.22 0.5633663193532185
0.23 0.560672398997907
0.24 0.5584215170032178
0.25 0.5561745367701423
0.26 0.5539275550780942
0.27 0.5516805765437588
0.28 0.5494335916156496
0.29 0.5471866128209126
0.3 0.5449396281895061
0.31 0.5426926532141354
0.32 0.5404456671990246
0.33 0.5381986871468771
0.34 0.5359517026583005
0.35000000000000003 0.5337047259630261
0.36 0.5314577402853825
0.37 0.5292107584912206
0.38 0.5269637785294261
0.39 0.5247167979437749
0.4 0.5224698153194
0.41000000000

### Case of m = 2

In [14]:
def reps2(x):
    x=set_vars(*x)
    dw=x.d*w(x)
    return 2* multiH(dw,[dw/2-x.o,dw/2-x.o,x.o])+2* multiH(dw,[(dw-x.t)/2,(dw-x.t)/2])+multiH(x.d*(1-4*w(x)),[x.z1,x.z1,x.z2,x.z2])

number_opt_params=5
m=2

set_vars = collections.namedtuple('CSIDH3', 't o z1 z2 d')
def csidh3(f) : return wrap(f, set_vars) 

In [15]:
w= lambda x: 1/5
a = lambda x: w(x)/2 +(x.t+ x.z1)/x.d
b = lambda x: w(x)/2 + (x.z2 -x.t/2+x.o)/x.d
D= lambda x: multiH(x.d,[x.d*a(x),x.d*a(x),x.d*b(x),x.d*b(x)])+multiH((1-x.d)/2,[(1-x.d)*w(x)/2,(1-x.d)*w(x)/2,(1-x.d)*w(x)/2,(1-x.d)*w(x)/2])
R = lambda x: reps2(x)
z0 = lambda x: (x.d*(1-2*w(x))-2*x.z1-2*x.z2 -2*x.o-x.t)/x.d
constraints_csidh = [
  { 'type' : 'eq',   'fun' : csidh3(lambda x : 1-z0(x)-2*a(x)-2*b(x))},
  { 'type' : 'ineq',   'fun' : csidh3(lambda x : x.d*w(x)/2-x.o)},
  { 'type' : 'ineq',   'fun' : csidh3(lambda x : x.d*w(x)/2-x.t/2)},
  { 'type' : 'ineq',   'fun' : csidh3(lambda x : x.d*(1-4*w(x))-2*x.z1-2*x.z2)},
  { 'type' : 'ineq',   'fun' : csidh3(lambda x : a(x))},
  { 'type' : 'ineq',   'fun' : csidh3(lambda x : b(x))},
  { 'type' : 'ineq',   'fun' : csidh3(lambda x : z0(x))},
]



In [16]:
PROCESSES = 40
ITERATIONS_PER_PROCESS=200

Lm2=compute_list(m)

0.0 0.6363016458566573
0.01 0.631309428544971
0.02 0.6262813565980068
0.03 0.6212814234803357
0.04 0.6163114724686573
0.05 0.6112813522185926
0.06 0.6062813764377059
0.07 0.6012813503929356
0.08 0.5962815085785174
0.09 0.591285146441815
0.1 0.5862813521770994
0.11 0.5812819671408844
0.12 0.5762863394690201
0.13 0.571281550456471
0.14 0.5662821651671623
0.15 0.5612813526199477
0.16 0.5562814377238661
0.17 0.5514939783372638
0.18 0.5483663783570181
0.19 0.5457033798718693
0.2 0.5430412870776813
0.21 0.5405677715494379
0.22 0.538425025353467
0.23 0.5362668236693673
0.24 0.5334888078238496
0.25 0.5319170208647013
0.26 0.5299523828486714
0.27 0.5280934233756829
0.28 0.5264376839676377
0.29 0.5251424677923161
0.3 0.5235742107365807
0.31 0.5224212553020554
0.32 0.5212006759242943
0.33 0.5200226897695371
0.34 0.5188356166793697
0.35000000000000003 0.5177017069978015
0.36 0.5164924868039096
0.37 0.5153191656016691
0.38 0.5141455517739776
0.39 0.5129591666888748
0.4 0.5118058407204379
0.41000000

### m=2 increased rep

In [17]:
def reps2(x):
    x=set_vars(*x)
    dw=x.d*w(x)
    return 2* multiH(dw,[dw/2-x.o-x.d1,dw/2-x.o-x.d1,x.o,x.o,x.d1])+2* multiH(dw,[(dw-x.t)/2-x.d2,(dw-x.t)/2-x.d2,x.t,x.d2])+multiH(x.d*(1-4*w(x)),[x.z1,x.z1,x.z2,x.z2,x.z3,x.z3])


In [18]:
set_vars = collections.namedtuple('CSIDH4', 't o z1 z2 z3 d1 d2 d')
def csidh4(f) : return wrap(f, set_vars) 

number_opt_params=8
m=2

In [19]:
w= lambda x: 1/5
a = lambda x: w(x)/2 +( x.z1 -x.d1 + x.d2 +x.t)/x.d
b = lambda x: w(x)/2 + (-x.t/2 - x.d2+x.z2 +x.o+x.d1)/x.d
c = lambda x: (x.z3 + x.d1 + x.d2)/x.d
D= lambda x: multiH(x.d,[x.d*a(x),x.d*a(x),x.d*b(x),x.d*b(x),x.d*c(x),x.d*c(x)])+multiH((1-x.d)/2,[(1-x.d)*w(x)/2,(1-x.d)*w(x)/2,(1-x.d)*w(x)/2,(1-x.d)*w(x)/2])
R = lambda x: reps2(x)
z0 = lambda x: (x.d*(1-2*w(x))-2*(x.z1+x.z2+x.z3+x.o+x.d1+x.d2+x.t/2))/x.d

constraints_csidh = [
  { 'type' : 'eq',   'fun' : csidh4(lambda x : 1-z0(x)-2*a(x)-2*b(x)-2*c(x))},
  { 'type' : 'ineq',   'fun' : csidh4(lambda x : w(x)/2-x.o-x.d1)},
  { 'type' : 'ineq',   'fun' : csidh4(lambda x : w(x)/2-x.t/2-x.d2)},
  { 'type' : 'ineq',   'fun' : csidh4(lambda x : x.d*(1-4*w(x))-2*x.z1-2*x.z2-2*x.z3)},
  { 'type' : 'ineq',   'fun' : csidh4(lambda x : a(x))},
  { 'type' : 'ineq',   'fun' : csidh4(lambda x : b(x))},
  { 'type' : 'ineq',   'fun' : csidh4(lambda x : c(x))},
  { 'type' : 'ineq',   'fun' : csidh4(lambda x : z0(x))},

]

In [20]:

#product should be at least 4000 to reach reasonable accuracy (better 40000)
PROCESSES = 40
ITERATIONS_PER_PROCESS=200

Lm2_increased_rep=compute_list(m)

0.0 0.62901993519702
0.01 0.6239392956749019
0.02 0.6189544908018265
0.03 0.6141958971556396
0.04 0.6090483060179404
0.05 0.6040218149332237
0.06 0.5989589087597853
0.07 0.5940474989676716
0.08 0.5888816156351613
0.09 0.5839173570302998
0.1 0.5788899486806973
0.11 0.5743544902613089
0.12 0.5690390304488265
0.13 0.5641578256999967
0.14 0.5594052613247159
0.15 0.5554177327054908
0.16 0.5521874053333423
0.17 0.5493660222968306
0.18 0.5472713609620798
0.19 0.5442322150109378
0.2 0.5414098756236669
0.21 0.5390265467141067
0.22 0.5370142214115511
0.23 0.535165288389486
0.24 0.5338936267230275
0.25 0.5312970797817699
0.26 0.5296610070021316
0.27 0.528688521936185
0.28 0.526881370346956
0.29 0.5251900334829265
0.3 0.5236870250045386
0.31 0.5224384832750721
0.32 0.5218863272232961
0.33 0.5208548619128627
0.34 0.5190748928425113
0.35000000000000003 0.5180594087083403
0.36 0.5164844046260609
0.37 0.5159967738568203
0.38 0.5144182831344639
0.39 0.5135388929395293
0.4 0.513321924190989
0.4100000000

### The case of m=3

In [21]:
number_opt_params=9
m=3


set_vars = collections.namedtuple('LWE', 't o z1 z2 z3 d1 d2 d3 d')
def csidh5(f) : return wrap(f, set_vars) 

def reps3(x):
    x=set_vars(*x)
    t=x.d*w(x)
    return 2* multiH(t,[t/2-x.o-x.d1,t/2-x.o-x.d1,x.o,x.o,x.d1])+2* multiH(t,[(t-x.t)/2-x.d2,(t-x.t)/2-x.d2,x.t,x.d2])\
            +2* multiH(t,[t/2-x.d3,t/2-x.d3,x.d3])\
            +multiH(x.d*(1-6*w(x)),[x.z1,x.z1,x.z2,x.z2,x.z3,x.z3])

In [22]:
w= lambda x: 1/7
a = lambda x: w(x)/2 +( x.z1 -x.d1 + x.d2 +x.t + x.d3)/x.d
b = lambda x: w(x)/2 + (-x.t/2 - x.d2+x.z2 +x.o+x.d1 + x.d3)/x.d
c = lambda x: w(x)/2 + (-x.d3 + x.z3 + x.d1 + x.d2)/x.d
D= lambda x: multiH(x.d,[x.d*a(x),x.d*a(x),x.d*b(x),x.d*b(x),x.d*c(x),x.d*c(x)])+multiH((1-x.d)/2,[(1-x.d)*w(x)/2,(1-x.d)*w(x)/2,(1-x.d)*w(x)/2,(1-x.d)*w(x)/2,(1-x.d)*w(x)/2,(1-x.d)*w(x)/2])
R = lambda x: reps3(x)

z0 = lambda x: (x.d*(1-3*w(x))-2*(x.z1+x.z2+x.z3+x.o+x.d1+x.d2+x.d3+x.t/2))/x.d
constraints_csidh = [
  { 'type' : 'eq',   'fun' : csidh5(lambda x : 1-z0(x)-2*a(x)-2*b(x)-2*c(x))},
  { 'type' : 'ineq',   'fun' : csidh5(lambda x : x.d*w(x)/2-x.o-x.d1)},
  { 'type' : 'ineq',   'fun' : csidh5(lambda x : x.d*w(x)/2-x.t/2-x.d2)},
  { 'type' : 'ineq',   'fun' : csidh5(lambda x : x.d*w(x)/2-x.d3)},
  { 'type' : 'ineq',   'fun' : csidh5(lambda x : x.d*(1-6*w(x))-2*x.z1-2*x.z2-2*x.z3)},
  { 'type' : 'ineq',   'fun' : csidh5(lambda x : a(x))},
  { 'type' : 'ineq',   'fun' : csidh5(lambda x : b(x))},
  { 'type' : 'ineq',   'fun' : csidh5(lambda x : c(x))},
  { 'type' : 'ineq',   'fun' : csidh5(lambda x : z0(x))},

]

In [23]:
#product should be at least 4000 to reach reasonable accuracy (better 40000)
PROCESSES = 40
ITERATIONS_PER_PROCESS=200

Lm3=compute_list(m)

0.0 0.6172254658939387
0.01 0.6121042371576131
0.02 0.6070684165563917
0.03 0.6020912739109244
0.04 0.5971372292829112
0.05 0.5923520751568091
0.06 0.5870773079616668
0.07 0.5821277134723702
0.08 0.5771553291695736
0.09 0.5721462265916413
0.1 0.5671563428757646
0.11 0.5620906108237634
0.12 0.5570922308891761
0.13 0.5520969383897905
0.14 0.5473094769359343
0.15 0.5428588280601815
0.16 0.5408753080858424
0.17 0.5385177262918327
0.18 0.5365225241470842
0.19 0.5346571039143221
0.2 0.5327777005427874
0.21 0.531438867328271
0.22 0.5299955880971223
0.23 0.5283883338151965
0.24 0.5274956019846505
0.25 0.5261843518384901
0.26 0.5252336403457463
0.27 0.5242298452906462
0.28 0.5230495382087832
0.29 0.5225433748031731
0.3 0.5214021972876033
0.31 0.5203235202202282
0.32 0.5189978023366175
0.33 0.517832797509757
0.34 0.5171807846603226
0.35000000000000003 0.5158776186996328
0.36 0.5146461809238191
0.37 0.5138889569759341
0.38 0.5127591816843726
0.39 0.511623505544768
0.4 0.5107145405586562
0.4100000

In [ ]:
from matplotlib import pyplot as plt
x,y =zip(*L_increased_rep)
xm2,ym2 =zip(*Lm2)
xm2_inc,xm2_inc=zip(*Lm2_increased_rep)
xm3,ym3 =zip(*Lm3)

plt.plot(x,y)
plt.plot(xm2,ym2)
plt.plot(xm2_inc,ym2_inc)
plt.plot(xm3,ym3)
plot.show()